## 01. 데이터세트 생성
* 2015~2022년 공공데이터포털에서 제공하는 원천데이터들을 병합,정제하여 분석을 위한 데이터세트로 적재합니다. 
* 2020년 데이터는 제공되지 않음.
    * 교육부에서 표본이 적어 제공하지 않음.
    * COVID 펜데믹으로 인해 건강검진표본이 적었을것으로 판단됨

### 1.1 원본 파일 불러오기
* 인코딩 방식이 연도별로 상이함
* 컬럼수가 상이하여, 병합을 위한 정제가 필요함

In [16]:
import pandas as pd
import numpy as np
import os

files = os.listdir(r'../data/org')
file_li = []

for f in files:
    file_dict = {}
    try : 
        df = pd.read_csv(fr'../data/org/{f}', encoding='euc-kr', low_memory=False, index_col=None)
    except : 
        df = pd.read_csv(fr'../data/org/{f}', encoding='utf-8', low_memory=False, index_col=None)

    
    file_dict = {'file_name': f, 'row_count' : df.shape[0], 'col_count' : df.shape[1], 'df' : df}
    file_li.append(file_dict)


### 1.2 필요없는 컬럼 삭제
* 2015~2017년도는 초등학생 데이터만 존재하는 관계로 초등학생 대상의 분석을 수행
* 중고등학생에게만 제공되는 정보의 컬럼은 삭제
    * 하루30분이상운동, 외상치료경험, 하루2시간이상게임_중고, 음란물채팅, 괴롭힘따돌림_중고, 가출생각_중고, 
    고민상담대상, 폭력위협, 학교문제상담희망_중고, 흡연음주전문가상담희망, 성문제전문가상담희망, 진로고민, 상담요청_중고, 상담희망_중고, 가출생각중고
    흡연_음주전문가상담희망, 괴롭힘따돌림중고, 음란물_채팅,  상담요청중고
* 특정 연도 자료에만 존재하는 필요없는(의미가 중요하지 않은) 컬럼, 중복된 내용의 컬럼은 삭제
    * ../data/schema 경로 내에 연도별 설명자료를 참조하여 판단
    * 학교ID, 학교명, 최종가중치, 도시규모별 분석용, 공학여부, 비뇨기, 개인ID, 1년동안치료경험, 순환기, 비뇨기, 호흡기, 간염검사, 층화변수(strata), 
    strata, 색각, 신경계, 소화기, 악관절이상, 광역시도, 게임시간

In [17]:
def drop_cols(df):
    df = df.drop(columns=['하루30분이상운동', '외상치료경험', '하루2시간이상게임_중고', '음란물채팅', '괴롭힘따돌림_중고',
                          '가출생각_중고', '고민상담대상', '폭력위협', '학교문제상담희망_중고', '흡연음주전문가상담희망', 
                          '성문제전문가상담희망', '진로고민', '상담요청_중고', '상담희망_중고', '가출생각중고', '학교ID', '학교명', '최종가중치',
                          '도시규모별 분석용', '공학여부', '흡연_음주전문가상담희망', '괴롭힘따돌림중고', '음란물_채팅', '비뇨기', '개인ID', '상담요청중고',
                          '1년동안치료경험', '순환기', '비뇨기', '호흡기', '간염검사', '층화변수(strata)', 'strata', '색각', '신경계', '소화기', '악관절이상',
                          '광역시도', '게임시간'
                          ],errors='ignore')
    return df

In [18]:
for i in range(len(file_li)):
    df = drop_cols(file_li[i].get('df'))
    file_li[i]['df'] = df
    file_li[i]['col_count'] = df.shape[1]


# file_li


### 1.2 공통함수 작성
* 컬럼정보를 비교하여 명칭을 일원화

In [19]:
def convert_col_rename(df):
    df = df.rename(columns={'아스파테이트아미노전이효소(AST U_L)' : 'AST(U_L)', '알라닌아미노전이효소(ALT U_L)' : 'ALT(U_L)', 'alt_UL' : 'ALT(U_L)',
                            '저밀도 지단백 콜레스테롤(LDL mg_dl)' : 'LDL(mg_dl)', 'ldl_mgdL' : 'LDL(mg_dl)', 'ldl(mg_dl)' : 'LDL(mg_dl)',
                            'hdl(mg_dl)' : 'HDL(mg_dl)', '고밀도 지단백 콜레스테롤(HDL mg_dl)' : 'HDL(mg_dl)', 'hdl_mgdL' : 'HDL(mg_dl)',
                            '몸무게' : '몸무게_kg', 
                            '시력_교정_좌_re': '시력_교정_좌', '하루2시간이상게임_초':'2시간이상게임',
                            '우유유제품': '우유_유제품', '시력_나안_좌_re' :  '시력_나안_좌',
                            '시력_교정_우_re' : '시력_교정_우', '결손치아영구치아__개수_상':  '결손치아(영구치아)__개수_상',
                            '이완기_mmHg' : '이완기', '채소김치제외':'채소(김치제외)',
                            '키':'키_cm',
                            '제3대구치사랑니' : '제3대구치(사랑니)','학교급별':'학교급',
                            '괴롭힘따돌림_초' : '괴롭힘따돌림', '혈당(식전)(mg_dl)':'혈당식전_mgdL',
                            '하루tv시청2시간이상' : '하루TV시청2시간이상', 'ast_UL': 'AST(U_L)',
                            '가출생각_초' : '가출생각', '시력_나안_우_re': '시력_나안_우',
                            '혈색소(g_dl)' : '혈색소_gdL',
                            '상담요청_초' : '상담요청',  '허리둘레' : '허리둘레_cm',
                            '자아신체상체형' : '자아신체상(체형)',  '중성지방_mgdL' : '중성지방(mg_dl)',
                            '치주질환잇몸병_유무' : '치주질환(잇몸병)_유무', '결손치아영구치아_유무' : '결손치아(영구치아)_유무',
                            '총콜레스테롤_mgdL' : '총콜레스테롤(mg_dl)', '수축기' :  '수축기_mmHg',
                            '결손치아영구치아__개수_하' : '결손치아(영구치아)__개수_하', '체질량지수' : '비만여부',
                            '상담희망_초':'상담희망',
                            '척추' : '근골격및 척추','우식발생위험치아_개수_하':'충치발생위험치아_개수_하',
                            '결핵흉부방사선검사' : '흉부방사선검사',  '우식치아_개수_하' : '충치치아_개수_하', '우식발생위험치아_개수_상' : '충치발생위험치아_개수_상',
                            '우식발생위험치아_유무' : '충치발생위험치아_유무', '제3대구치' : '제3대구치(사랑니)', '우식치아_유무' : '충치치아_유무',
                            '우식치아_개수_상' : '충치치아_개수_상', '치주질환_유무' : '치주질환(잇몸병)_유무',
                            '근골격및척추' : '근골격및 척추', '학교급별':'학교급', '시도' : '시도별'
                            })
    return df

* 동일 용도의 컬럼에 대한 하나의 컬럼으로 변환

In [20]:
import re

def update_col_value(f, df): 
        if '도시규모별 분석용' in df.columns:
                df['도시규모'] = df['도시규모별 분석용']
                df = df.drop(columns='도시규모별 분석용', errors='ignore')


        if '상담희망' not in df.columns:
                df['상담희망'] = None
        check_col_names = ['학교문제상담희망_초','가정문제걱정']
        if pd.Series(check_col_names).isin(df.columns).all():
                df.loc[((df['상담희망'].isna() == True) | (df['상담희망']== 2)) & ((df['학교문제상담희망_초']==1) | (df['가정문제걱정']==1)), '상담희망'] = 1
                df = df.drop(columns=check_col_names, errors='ignore')
        if '고민상담희망' in df.columns:
                df.loc[((df['상담희망'].isna() == True) | (df['상담희망']== 2)) & (df['고민상담희망']==1), '상담희망'] = 1
                df = df.drop(columns='고민상담희망', errors='ignore')
        

        if '학년도' not in df.columns:
                df['학년도'] = f[f.index('20'):4]
        return df

### 1.3 데이터 병합
* 2015~2022년도 데이터 merge
* 2020년도는 검사 샘플이 제공되지 않음(COVID로 인한 검사 시행 표본 수가 적음)

In [21]:
df = pd.DataFrame([])

for i in range(len(file_li)):
    f = file_li[i].get('file_name')
    df1 = convert_col_rename(file_li[i].get('df'))
    df1 = update_col_value(f, df1)

    df = df.reset_index(drop = True)
    df1 = df1[df1['학교급'] == '초'].reset_index(drop = True) # 초등학급 정보만 추출

    df = pd.concat([df1, df], ignore_index=True, axis=0, join='outer')
    print(df1.shape, df.shape)


(33424, 95) (33424, 95)
(37207, 96) (70631, 96)
(38448, 96) (109079, 97)
(33095, 95) (142174, 97)
(39093, 91) (181267, 97)
(39108, 91) (220375, 97)
(33342, 95) (253717, 97)


### 1.4 데이터 병합 확인

In [22]:
df.head()

,학년도,시도별,학교급,학년,반,순번,성별,생년월일,키_cm,몸무게_kg,...,상담요청,가족흡연,가족음주,무기력감,수업태도교정,과잉행동,주의력산만,상담희망,도시규모,가정문제걱정
0,2016,서울특별시교육청,초,1,1,1,남,20090512,130.2,26.5,...,2,2,2,2,2,2,2,None,NaN,NaN
1,2016,서울특별시교육청,초,1,1,2,남,20090819,123.1,27.7,...,2,2,2,2,2,2,2,None,NaN,NaN
2,2016,서울특별시교육청,초,1,1,3,남,20091021,117.2,19.1,...,2,2,2,2,2,2,2,None,NaN,NaN
3,2016,서울특별시교육청,초,1,1,4,남,20090413,123.5,22.2,...,2,2,2,2,2,2,2,None,NaN,NaN
4,2016,서울특별시교육청,초,1,1,5,남,20091007,123.5,22.2,...,2,1,1,2,2,2,2,None,NaN,NaN


In [23]:
# 표시할 최대 행 수를 None으로 설정하여 모든 행을 출력
pd.set_option('display.max_rows', None)
df.groupby(['학년도']).count().T

학년도,2015,2016,2017,2021,2022,2018,2019
시도별,33424,33342,33095,38448,37207,39108,39093
학교급,33424,33342,33095,38448,37207,39108,39093
학년,33424,33342,33095,38448,37207,39108,39093
반,33424,33342,33095,38448,37207,39108,39093
순번,33424,33342,33095,38448,37207,39108,39093
성별,33424,33342,33095,38448,37207,39108,39093
생년월일,33424,33342,33095,38448,37207,39108,39093
키_cm,33424,33342,33095,38448,37207,39091,39093
몸무게_kg,33424,33342,33095,38448,37207,39091,39093
비만여부,33424,33342,33095,0,37207,0,0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253717 entries, 0 to 253716
Data columns (total 97 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   학년도               253717 non-null  object 
 1   시도별               253717 non-null  object 
 2   학교급               253717 non-null  object 
 3   학년                253717 non-null  int64  
 4   반                 253717 non-null  int64  
 5   순번                253717 non-null  int64  
 6   성별                253717 non-null  object 
 7   생년월일              253717 non-null  object 
 8   키_cm              253700 non-null  float64
 9   몸무게_kg            253700 non-null  float64
 10  비만여부              137068 non-null  object 
 11  건강검진일             253717 non-null  object 
 12  근골격및 척추           176053 non-null  object 
 13  시력_나안_좌           167560 non-null  object 
 14  시력_나안_우           167562 non-null  object 
 15  시력_교정_좌           144032 non-null  object 
 16  시력_교정_우           14

# 1.5 데이터 1차 정제

* 원본 데이터에서 이상 데이터에 대한 기본적인 처리 진행
* 연도별 값이 공통적으로 존재하지 않는 컬럼은 일단 삭제하고 분석할 데이터세트를 생성함
    * HDL(mg_dl), 중성지방(mg_dl), LDL(mg_dl), 혈색소_gdL, 흉부방사선검사, 허리둘레_cm, 턱관절이상, 도시규모, 가정문제걱정, 반, 순번

In [25]:
df = df.drop(columns=['HDL(mg_dl)', '중성지방(mg_dl)', 'LDL(mg_dl)', '혈색소_gdL', '흉부방사선검사', '허리둘레_cm', '턱관절이상', '도시규모', '가정문제걱정', '반', '순번'],errors='ignore')

* 컬럼별 데이터 확인
    * 항목별 count를 참고했을때 연도별(2015~2017과 그 이외해의 데이터 건수 참고)로 널값을 표현하는 형식이 다를 것으로 예상됨
    * 이점을 고려하여 분석시 활용필요

In [26]:
df.groupby(['학년도']).count().T

학년도,2015,2016,2017,2021,2022,2018,2019
시도별,33424,33342,33095,38448,37207,39108,39093
학교급,33424,33342,33095,38448,37207,39108,39093
학년,33424,33342,33095,38448,37207,39108,39093
성별,33424,33342,33095,38448,37207,39108,39093
생년월일,33424,33342,33095,38448,37207,39108,39093
키_cm,33424,33342,33095,38448,37207,39091,39093
몸무게_kg,33424,33342,33095,38448,37207,39091,39093
비만여부,33424,33342,33095,0,37207,0,0
건강검진일,33424,33342,33095,38448,37207,39108,39093
근골격및 척추,33424,33342,33095,12249,12257,38887,12799


* 데이터 컬럼명 확인

In [27]:
df.columns

Index(['학년도', '시도별', '학교급', '학년', '성별', '생년월일', '키_cm', '몸무게_kg', '비만여부',
       '건강검진일', '근골격및 척추', '시력_나안_좌', '시력_나안_우', '시력_교정_좌', '시력_교정_우', '안질환',
       '청력_좌', '청력_우', '귓병', '콧병', '목병', '피부병', '요단백', '요잠혈', '혈당식전_mgdL',
       '총콜레스테롤(mg_dl)', 'AST(U_L)', 'ALT(U_L)', '수축기_mmHg', '이완기', '기타',
       '건강검진_종합소견', '충치치아_유무', '충치치아_개수_상', '충치치아_개수_하', '충치발생위험치아_유무',
       '충치발생위험치아_개수_상', '충치발생위험치아_개수_하', '결손치아(영구치아)_유무', '결손치아(영구치아)__개수_상',
       '결손치아(영구치아)__개수_하', '구내염및연조직질환', '부정교합', '구강위생상태', '그밖의치아상태',
       '치주질환(잇몸병)_유무', '치주질환_종류', '치아마모증', '제3대구치(사랑니)', '구강검진_종합소견', '구강검진일',
       '라면', '음료수', '패스트푸드', '육류', '우유_유제품', '과일', '채소(김치제외)', '아침식사',
       '다이어트경험_답변1', '다이어트경험_답변2', '다이어트경험_답변3', '다이어트경험_답변4', '주3회이상운동',
       '하루수면량', '자아신체상(체형)', '손씻기', '양치질', '안전벨트착용', '안전장비착용', '하루TV시청2시간이상',
       '2시간이상게임', '괴롭힘따돌림', '현금갈취', '신체접촉', '가출생각', '가족지지', '체벌경험', '상담요청',
       '가족흡연', '가족음주', '무기력감', '수업태도교정', '과잉행동', '주의력산만', '상담희망'],
      dtype='object')

#### 세부 데이터 확인 및 정제
* 학년도 컬럼 확인

In [28]:
df['학년도'] = df['학년도'].astype(int)
df['학년도'].value_counts(dropna=False)

학년도
2018    39108
2019    39093
2021    38448
2022    37207
2015    33424
2016    33342
2017    33095
Name: count, dtype: int64

* 시도별 컬럼 확인

In [29]:
# df['시도별'].unique()

서울, 부산, 대구, 인천, 광주, 대전, 울산, 세종, 경기, 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

In [30]:

df.loc[df['시도별'].str.startswith('서울'), '시도별'] = '서울'
df.loc[df['시도별'].str.startswith('제주'), '시도별'] = '제주'
df.loc[df['시도별'].str.startswith('서귀포'), '시도별'] = '제주'
df.loc[df['시도별'].str.startswith('경상북'), '시도별'] = '경북'
df.loc[df['시도별'].str.startswith('경상남'), '시도별'] = '경남'
df.loc[df['시도별'].str.startswith('인천'), '시도별'] = '인천'
df.loc[df['시도별'].str.startswith('충청남'), '시도별'] = '충남'
df.loc[df['시도별'].str.startswith('충청북'), '시도별'] = '충북'
df.loc[df['시도별'].str.startswith('전라북'), '시도별'] = '전북'
df.loc[df['시도별'].str.startswith('전라남'), '시도별'] = '전남'
df.loc[df['시도별'].str.startswith('부산'), '시도별'] = '부산'
df.loc[df['시도별'].str.startswith('대구'), '시도별'] = '대구'
df.loc[df['시도별'].str.startswith('경기'), '시도별'] = '경기'
df.loc[df['시도별'].str.startswith('광주'), '시도별'] = '광주'
df.loc[df['시도별'].str.startswith('대전'), '시도별'] = '대전'
df.loc[df['시도별'].str.startswith('울산'), '시도별'] = '울산'
df.loc[df['시도별'].str.startswith('세종'), '시도별'] = '세종'
df.loc[df['시도별'].str.startswith('강원'), '시도별'] = '강원'
df['시도별'].value_counts(dropna=False)

시도별
경기    40271
서울    26845
경북    19293
경남    18652
인천    15615
전남    14325
부산    14207
전북    12915
대구    12610
충남    12589
강원    12266
충북    11407
울산    10477
대전     9092
광주     8980
제주     7829
세종     6344
Name: count, dtype: int64

* 학교급 데이터 확인

In [31]:
df['학교급'].value_counts(dropna=False)

학교급
초    253717
Name: count, dtype: int64

* 학년 데이터 확인

In [32]:
df['학년'].value_counts(dropna=False)

학년
4    42536
2    42336
5    42332
3    42317
6    42241
1    41955
Name: count, dtype: int64

* 생년월일 확인
    * 널값이거나 유효하지 않은 값이 있는 경우가 있으나 분석에 활용시점에 변환처리 예정

In [33]:
df['생년월일'] = df['생년월일'].str.replace('-', '')
df['생년월일'].value_counts(dropna=False)

생년월일
NaN         216510
20150102        57
20140103        51
20110103        44
20120102        43
20150105        41
20100106        38
20130108        37
20110113        37
20100928        36
20120103        36
20140120        36
20110101        36
20130111        35
20140102        35
20150103        35
20110118        34
20110105        34
20150116        34
20130116        34
20101012        33
20130102        33
20130924        33
20120106        33
20110107        33
20110125        33
20120927        32
20120227        32
20130923        32
20120105        32
20140126        31
20100109        31
20130107        31
20100120        31
20100114        31
20110119        31
20120419        31
20110112        31
20130110        31
20131007        31
20110902        31
20140107        31
20150223        31
20100208        31
20151201        31
20100101        30
20130122        30
20130104        30
20150807        30
20110523        30
20110108        30
20150109        30
2012042

* 성별 확인

In [34]:
df['성별'].value_counts(dropna=False)

성별
남    130597
여    123120
Name: count, dtype: int64

* 키, 몸무게 확인

In [35]:
check_columns = ['키_cm', '몸무게_kg']
df[check_columns].describe()

,키_cm,몸무게_kg
count,253700.000000,253700.000000
mean,136.730823,35.508370
std,12.169221,11.695431
min,92.000000,10.000000
25%,127.200000,26.500000
50%,136.100000,33.100000
75%,145.800000,42.200000
max,218.000000,150.000000


* 건강검진일 확인

In [36]:
df.loc[df['건강검진일'].isin(['정상(경계)', '정밀검사요함']), '건강검진일'] = '99999999'
df['건강검진일'] = df['건강검진일'].str.replace('-', '')
df['건강검진일'].fillna(0).astype(int).astype(str)
df['건강검진일'].unique()

array([nan, '20170420', '20170425', '20170421', '20170418', '20170422',
       '20170427', '20170413', '20170412', '20170417', '20170424',
       '20170530', '20170525', '20170529', '20170502', '20170511',
       '20170515', '20170504', '20170531', '20170512', '20170616',
       '20170516', '20170517', '20170608', '20170519', '20170602',
       '20170523', '20170607', '20170522', '20170615', '20170612',
       '20170510', '20170613', '20170601', '20170605', '20170524',
       '20170614', '20170526', '20170509', '20170419', '20170426',
       '20170503', '20170429', '20170506', '20170518', '20170609',
       '20170711', '20170501', '20170428', '20170513', '20170414',
       '20170603', '20170508', '20170520', '20170404', '20170407',
       '20170620', '20170527', '20170619', '20170610', '20170623',
       '20170703', '20170705', '20170627', '20170626', '20170629',
       '20170628', '20170624', '20170621', '20170617', '20170701',
       '20170323', '20170324', '20170321', '20170320', '2

* 그 밖의 문자형 컬럼 정제

In [37]:
df['충치발생위험치아_유무'] = df['충치발생위험치아_유무'].str.replace('0', '무')
df['결손치아(영구치아)_유무'] = df['결손치아(영구치아)_유무'].str.replace('0', '무')
df['구내염및연조직질환'] = df['구내염및연조직질환'].str.replace('0', '무')

In [38]:
df['부정교합'] = df['부정교합'].str.replace('무', '없음')

In [39]:
def convert_object_cols(col_name):
    df[col_name] = df[col_name].str.strip().replace('', np.nan)
    df[col_name].value_counts(dropna=False)


col_name_list = ['근골격및 척추', '안질환', '청력_좌', '청력_우', '비만여부', '귓병', '콧병', '목병', 
                 '피부병', '요단백', '요잠혈', '기타', '건강검진_종합소견', '충치치아_유무', '충치발생위험치아_유무', '결손치아(영구치아)_유무',
                 '구내염및연조직질환', '부정교합', '구강위생상태', '그밖의치아상태',  '치주질환(잇몸병)_유무', '치주질환_종류',
                 '치아마모증', '제3대구치(사랑니)', '구강검진_종합소견', '구강검진일']
for col_name in col_name_list:
    convert_object_cols(col_name)
    print(df[col_name].value_counts(dropna=False))
    print()

근골격및 척추
NaN     170300
정상       83031
척추측만       203
검사안함       112
기타          60
요통           8
척추전만         2
어깨결림         1
Name: count, dtype: int64

안질환
NaN     169908
없음       83410
검사안함       233
기타          80
사시          49
결막염         37
Name: count, dtype: int64

청력_좌
NaN     169916
정상       83648
이상          81
검사안함        72
Name: count, dtype: int64

청력_우
NaN     169917
정상       83629
이상         100
검사안함        71
Name: count, dtype: int64

비만여부
NaN     116695
정상       72507
정상체중     24184
비만       17039
과체중      16901
저체중       6391
Name: count, dtype: int64

귓병
NaN         169895
없음           83673
중이염             56
기타              37
검사안함            34
외이도염            16
중이염및외이도염         6
Name: count, dtype: int64

콧병
NaN     169895
없음       79584
비염        3965
기타         120
부비동염        93
검사안함        32
코곁굴염        28
Name: count, dtype: int64

목병
NaN        169930
없음          81633
편도비대         2035
기타             69
검사안함           32
경부림프절종대        14
갑상선비대    

* 그 밖의 숫자형이어야 하는 컬럼 정제

In [40]:
df['혈당식전_mgdL'] = df['혈당식전_mgdL'].replace('음성', np.nan)

In [41]:
df['충치치아_개수_상'] = df['충치치아_개수_상'].replace('무', np.nan)
df['충치발생위험치아_개수_상'] = df['충치발생위험치아_개수_상'].replace('무', np.nan)
df['결손치아(영구치아)__개수_상']= df['결손치아(영구치아)__개수_상'].replace('무', np.nan)

In [42]:
df['라면'] = df['라면'].replace(['', ' '], np.nan)
df.loc[df['라면'].astype(float)>1000, '라면'] = None

In [43]:
def convert_number_cols(col_name):
    df[col_name] = df[col_name].replace(['', ' '], np.nan)
    df[col_name] = df[col_name].astype('float')

col_name_list = ['시력_나안_좌', '시력_나안_우', '시력_교정_좌', '시력_교정_우', '혈당식전_mgdL', '총콜레스테롤(mg_dl)', 
                 'AST(U_L)', 'ALT(U_L)', '수축기_mmHg', '이완기', '충치치아_개수_상', '충치치아_개수_하', '충치발생위험치아_개수_상',
                 '충치발생위험치아_개수_하', '결손치아(영구치아)__개수_상', '결손치아(영구치아)__개수_하', '라면', '음료수', '패스트푸드',
                 '육류', '우유_유제품', '과일', '채소(김치제외)', '아침식사', '다이어트경험_답변1', '다이어트경험_답변2',
                 '다이어트경험_답변3', '다이어트경험_답변4', '주3회이상운동', '하루수면량', '자아신체상(체형)', '손씻기',
                 '양치질', '안전벨트착용', '안전장비착용', '하루TV시청2시간이상', '2시간이상게임', '괴롭힘따돌림',
                 '현금갈취', '신체접촉', '가출생각', '가족지지', '체벌경험', '상담요청', '가족흡연', '가족음주', '무기력감',
                 '수업태도교정', '과잉행동', '주의력산만', '상담희망']

for col_name in col_name_list:
    convert_number_cols(col_name)
    print(col_name)
    print(df[col_name].unique())
    print()



시력_나안_좌
[0.4               nan 1.         0.5        0.6        0.2
 0.8        0.3        0.1        1.5        0.9        1.2
 0.7        2.         0.         1.6        1.3        1.9
 1.7        0.15       1.4        1.25       0.63       0.32
 0.25       1.8        1.1        0.69999999 0.89999998 0.80000001
 0.60000002 0.30000001 0.40000001 0.15000001 0.06       0.12
 0.16      ]

시력_나안_우
[0.8               nan 1.2        1.         0.3        0.6
 0.5        0.4        0.2        0.1        0.9        1.5
 2.         0.7        0.         1.6        1.1        1.3
 1.8        0.15       0.63       1.25       0.25       0.32
 1.9        0.89999998 0.69999999 0.80000001 0.60000002 0.40000001
 0.30000001 0.15000001 1.05       0.07       0.12       0.16
 1.21       1.62      ]

시력_교정_좌
[       nan 0.5        0.6        0.8        1.         0.4
 0.2        1.2        1.5        0.7        0.9        0.
 0.3        0.1        2.         0.15       0.89999998 0.69999999
 0.80000001 0

/var/folders/jw/wdnx0tjx7kzdqt8y2gcwtz940000gn/T/ipykernel_1163/1893871733.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col_name] = df[col_name].replace(['', ' '], np.nan)


* 데이터 정제 결과 확인

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253717 entries, 0 to 253716
Data columns (total 86 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   학년도               253717 non-null  int64  
 1   시도별               253717 non-null  object 
 2   학교급               253717 non-null  object 
 3   학년                253717 non-null  int64  
 4   성별                253717 non-null  object 
 5   생년월일              37207 non-null   object 
 6   키_cm              253700 non-null  float64
 7   몸무게_kg            253700 non-null  float64
 8   비만여부              137022 non-null  object 
 9   건강검진일             70302 non-null   object 
 10  근골격및 척추           83417 non-null   object 
 11  시력_나안_좌           68537 non-null   float64
 12  시력_나안_우           68549 non-null   float64
 13  시력_교정_좌           11984 non-null   float64
 14  시력_교정_우           11979 non-null   float64
 15  안질환               83809 non-null   object 
 16  청력_좌              83

### 1.6 병합 데이터 저장
* 분석을 위한 연도별 건강검사 원자료들의 병합데이터를 저장해 놓는다.
* 추후 분석용 코드에서 분석을 위한 가공을 진행한다.

In [45]:
df.to_csv(fr'../data/input/data.csv', encoding='utf-8', index=False)